# Exercise 1 - Building a 0-shot classifier with Outlines

In this notebook we will take our first look at how to use [Outlines](https://github.com/outlines-dev/outlines). Most of the code is written for you, but you will be required to implement small snippets of code to make sure you understand the key concepts being discussed.




In [2]:
import json
import outlines
import torch
from transformers import AutoTokenizer
from textwrap import dedent

Here we load our model using `outlines.models.transformers`.

**Note**: If you are not using an Apple Silicon device, you should change `mps` to either `cuda` if you have an NVIDIA GPU or `cpu` otherwise.

In [3]:
model_name = "Qwen/Qwen2-0.5B-Instruct"
model = outlines.models.transformers(
    model_name,
    device='mps',
    model_kwargs={
        'torch_dtype': torch.bfloat16,
        'trust_remote_code': True
    })
tokenizer = AutoTokenizer.from_pretrained(model_name)

## The Task: Predicting Department

In this example we'll be processing messages that have received involving user complaints, here is the data loaded with a few examples:

In [5]:
with open("../examples.json",'r') as fin:
    complaint_data = json.loads(fin.read())
complaint_data[0:3]

[{'message': 'Hi, my name is Olivia Brown.I recently ordered a knife set from your wellness range, and it arrived earlier this week. Unfortunately, my satisfaction with the product has been less than ideal.My order was A123456',
  'order_number': 'A12-3456',
  'department': 'kitchen'},
 {'message': 'Hi, my name is John Smith.I recently ordered a dress for an upcoming event, which was alleged to meet my expectations both in fit and style. However, upon arrival, it became apparent that the fabric was of subpar quality, leading to a less than satisfactory appearance.The order number is A12-3456',
  'order_number': 'A12-3456',
  'department': 'clothing'},
 {'message': 'Hi, my name is Sarah Johnson.I recently ordered the ultimate ChefMaster 8 Drawer Cooktop. However, upon delivery, I discovered that one of the burners is malfunctioning.My order was A458739',
  'order_number': 'A45-8739',
  'department': 'kitchen'}]

As you can see, there are departments associated with each complaint. What we want to do is build a **zero-shot** classifier to predcit the department given the content of the `message`. Here are the departments in our example:

In [6]:
departments = ["clothing","electronics","kitchen","automotive"]

Zero-shot classifiers are great use cases for LLMs because they allow us to try to classify data with *zero* example cases. This is particularly helpful when we don't have any existing data to train a traditional machine learning classifier on. For the purpose of this workshop we're using defaulting to the small but not very powerful `Qwen/Qwen2-0.5B-Instrct` model. This is to ensure this model runs reasonably well on most laptops. This model performs relatively poorly on the task, but if we were to upgrade to something like `Phi-3-medium` we can get 92% accuracy on this task *with no traning examples required!*.

### Our Prompt

We are going to be using an `instruct` model which means we can use a *instruction* or *chat-based* style of prompting. The function below takes our complaint and formats it into a prompt to send to the LLM. Note the use of `tokenizer.apply_chat_template`. Each instruct LLM has a slightly different format for it's instruction prompts, but using `apply_chat_template` makes it easy to generalize.

In [6]:
def create_prompt(complaint):
    prompt_messages = [
        {
            "role": "system",
            "content": "You are as agent designed to help label complaints."
        },
        {
        "role": "user",
        "content": dedent("""
        I'm going to provide you with a consumer complaint to analyze.
        The complaint is going to be regarding a product from one of our
        departments. Here is the list of departments:
            - "clothing"
            - "electronics"
            - "kitchen"
            - "automotive"
        Please reply with *only* the name of the department.
        """)
    },{
        "role": "assistant",
        "content": "I understand and will only answer with the department name"
    },{
        "role": "user",
        "content": f"Great! Here is the complaint: {complaint['message']}"
    }
                       
                      ]
    prompt = tokenizer.apply_chat_template(prompt_messages, tokenize=False)
    return prompt

This is how the first complaint looks when rendered as a prompt for our model:

In [7]:
create_prompt(complaint_data[0])

'<|im_start|>system\nYou are as agent designed to help label complaints.<|im_end|>\n<|im_start|>user\n\nI\'m going to provide you with a consumer complaint to analyze.\nThe complaint is going to be regarding a product from one of our\ndepartments. Here is the list of departments:\n    - "clothing"\n    - "electronics"\n    - "kitchen"\n    - "automotive"\nPlease reply with *only* the name of the department.\n<|im_end|>\n<|im_start|>assistant\nI understand and will only answer with the department name<|im_end|>\n<|im_start|>user\nGreat! Here is the complaint: Hi, my name is Olivia Brown.I recently ordered a knife set from your wellness range, and it arrived earlier this week. Unfortunately, my satisfaction with the product has been less than ideal.My order was A123456<|im_end|>\n'

## Unstructured Generation

To demonstrate the value of *structured* generation we'll first start with an *unstructured* example. We can still use Outlines for this! In this case we'll use `outlines.generate.text` to create a text generator using our LLMs.

In [7]:
generator = outlines.generate.text(model)

Next we iterate through a few examples to see what our unstructured response looks like:

In [9]:
for complaint in complaint_data[0:4]:
    prompt = create_prompt(complaint)
    result = generator(prompt, max_tokens=12)
    print(f"LLM: {result} Actual: {complaint['department']}")

We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


LLM: assistant
My apologies, my system does not recognize this Actual: kitchen
LLM: assistant
Department: "clothing" Actual: clothing
LLM: assistant
The department with which the complaint related is: Actual: kitchen
LLM: assistant
electronics Actual: electronics


As you can see the results are a bit of mess! In this case we'll need to do a bit of additional parse to out the actual answer (if there is one).

Using Structured Generation with Outlines we no longer have to worry about this.

## Structured Generation

This first exercise is quite simple, you are going to use `outlines.generate.choice` to create a structured generator that will product only structured text. Replace the:

```
generator_struct = None
```

With the correct code to solve this problem with `choice`. This task should be very easy, but it's a good first start at using Outlines.

In [10]:
correct = 0
generator_struct = outlines.generate.choice(model,departments)
for complaint in complaint_data:
    prompt = create_prompt(complaint)
    result = generator_struct(prompt)
    if result == complaint['department']:
        correct += 1
    print(f"LLM: {result} Actual: {complaint['department']}")
print(correct/len(complaint_data))

LLM: electronics Actual: kitchen
LLM: electronics Actual: clothing
LLM: electronics Actual: kitchen
LLM: electronics Actual: electronics
LLM: electronics Actual: electronics
LLM: clothing Actual: kitchen
LLM: electronics Actual: electronics
LLM: electronics Actual: automotive
LLM: electronics Actual: electronics
LLM: electronics Actual: kitchen
LLM: electronics Actual: automotive
LLM: electronics Actual: automotive
LLM: electronics Actual: kitchen
LLM: electronics Actual: kitchen
LLM: electronics Actual: automotive
LLM: electronics Actual: kitchen
LLM: electronics Actual: automotive
LLM: electronics Actual: kitchen
LLM: electronics Actual: kitchen
LLM: electronics Actual: clothing
LLM: electronics Actual: electronics
LLM: electronics Actual: automotive
LLM: electronics Actual: electronics
LLM: electronics Actual: kitchen
LLM: electronics Actual: kitchen
LLM: electronics Actual: clothing
LLM: clothing Actual: kitchen
LLM: electronics Actual: clothing
LLM: clothing Actual: kitchen
LLM: e

Depending on the underlying LLM you use, you may get dramatically different performance but in all cases you should find that the model *always* produces just a single, valid department name as an option.